In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
import json
import os
import numpy as np
import shutil # For sorting files

In [2]:
!pip install kaggle



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!kaggle --version


Kaggle API 1.7.4.5


In [6]:
os.makedirs("../data/user2", exist_ok=True)


In [7]:


# Download competition dataset into that folder
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition -p ../data/user2

[Errno 28] No space left on device



  0%|          | 0.00/814M [00:00<?, ?B/s]
  0%|          | 1.00M/814M [00:01<13:41, 1.04MB/s]
  2%|▏         | 17.0M/814M [00:01<00:38, 21.5MB/s]
  3%|▎         | 25.0M/814M [00:01<00:37, 22.1MB/s]


^C



  0%|          | 0.00/814M [00:00<?, ?B/s]
  0%|          | 1.00M/814M [00:00<03:59, 3.55MB/s]
  2%|▏         | 20.0M/814M [00:00<00:12, 66.5MB/s]
  5%|▍         | 39.0M/814M [00:00<00:07, 108MB/s] 
  7%|▋         | 61.0M/814M [00:00<00:05, 144MB/s]
 10%|▉         | 79.0M/814M [00:00<00:04, 157MB/s]
 12%|█▏        | 97.0M/814M [00:00<00:04, 162MB/s]
 14%|█▍        | 117M/814M [00:00<00:04, 175MB/s] 
 17%|█▋        | 135M/814M [00:01<00:14, 50.6MB/s]
 19%|█▉        | 153M/814M [00:01<00:10, 65.0MB/s]
 21%|██        | 171M/814M [00:02<00:08, 81.2MB/s]
 23%|██▎       | 187M/814M [00:02<00:07, 92.7MB/s]
 26%|██▌       | 211M/814M [00:02<00:05, 121MB/s] 
 28%|██▊       | 229M/814M [00:02<00:04, 126MB/s]
 31%|███       | 250M/814M [00:02<00:04, 145MB/s]
 33%|███▎      | 272M/814M [00:02<00:03, 163MB/s]
 36%|███▌      | 291M/814M [00:02<00:03, 150MB/s]
 38%|███▊      | 308M/814M [00:04<00:12, 41.8MB/s]
 41%|████      | 333M/814M [00:04<00:08, 59.9MB/s]
 44%|████▍     | 356M/814M [00:04<00:06

In [11]:
import zipfile
from pathlib import Path

# unzip main file
main_zip = Path("../data/user2/dogs-vs-cats-redux-kernels-edition.zip")
with zipfile.ZipFile(main_zip, 'r') as z:
    z.extractall("../data/user2")

# unzip training set
train_zip = Path("../data/user2/train.zip")
with zipfile.ZipFile(train_zip, 'r') as z:
    z.extractall("../data/user2/train_images")


In [12]:

print("Sorting User 2's images...")

RAW_DATA_DIR = '../data/user2/train_images/train'
SORTED_DATA_DIR = '../data/user2/train_sorted'

# Create class folders if they don't exist
os.makedirs(os.path.join(SORTED_DATA_DIR, 'cats'), exist_ok=True)
os.makedirs(os.path.join(SORTED_DATA_DIR, 'dogs'), exist_ok=True)

# List files
all_files = os.listdir(RAW_DATA_DIR)
print(f"Found {len(all_files)} files in train_images")

for f in all_files:
    src = os.path.join(RAW_DATA_DIR, f)

    # Skip directories
    if os.path.isdir(src):
        continue

    fname = f.lower()  # make case-insensitive

    if fname.startswith('cat'):
        shutil.move(src, os.path.join(SORTED_DATA_DIR, 'cats', f))
    elif fname.startswith('dog'):
        shutil.move(src, os.path.join(SORTED_DATA_DIR, 'dogs', f))
    else:
        print("⚠️ Unknown file skipped:", f)

print("Image sorting complete.")

Sorting User 2's images...
Found 25000 files in train_images
Image sorting complete.


In [13]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
DATA_DIR_USER2 = SORTED_DATA_DIR # Use the new sorted directory

datagen_u2 = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator_u2 = datagen_u2.flow_from_directory(
    DATA_DIR_USER2,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary', # Changed to binary
    subset='training'
)
validation_generator_u2 = datagen_u2.flow_from_directory(
    DATA_DIR_USER2,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary', # Changed to binary
    subset='validation'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [16]:
# --- ML Code: Test Model V1 ---
print("Testing model_v1 on User 2's validation data...")
try:
    model_v1 = tf.keras.models.load_model('../models/model_v1.h5')
    print("Loaded model_v1.h5")

    results_v1_on_u2_data = model_v1.evaluate(validation_generator_u2)

    test_metrics_v1_u2 = {
        'test_loss': results_v1_on_u2_data[0],
        'test_accuracy': results_v1_on_u2_data[1]
    }
except Exception as e:
    print(f"Error testing model_v1: {e}")
    test_metrics_v1_u2 = {'test_loss': 'Error', 'test_accuracy': 'Error'}

os.makedirs('../results', exist_ok=True)

# Save test results
with open('../results/test_v1_user2.json', 'w') as f:
    json.dump(test_metrics_v1_u2, f, indent=4)

print("Saved test_v1_user2.json to results/")

print("✅ Phase 2, Step 4: Testing of model_v1 complete.")


Testing model_v1 on User 2's validation data...


Loaded model_v1.h5
157/157 ━━━━━━━━━━━━━━━━━━━━ 31s 189ms/step - accuracy: 0.7066 - loss: 0.5699
Saved test_v1_user2.json to results/
✅ Phase 2, Step 4: Testing of model_v1 complete.
